In [289]:
# import the required modules
import numpy as np
import random as random
import math as math
import copy as copy

In [290]:
# a class built to handle playing BlackJack
class BlackJack():
    
    # useful constants
    cardValues = {1: 'Ace', 11: 'Jack', 12: 'Queen', 13: 'King'}
    cardSuits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    
    def __init__(self, cards):
        # the state of the game: my cards, the dealer's cards, and our scores
        self.state = {
            "myCards" : [],
            "dealerCards" : [],     
            "myScore" : 0,
            "dealerScore": 0
        }
        # who won, 0 for in progress, -1 for tie, 1 for you win, 2 for the dealer wins
        self.winner = 0
        self.deck = copy.deepcopy(cards)
    
    # calculates the score of someone's hand
    def score(self, cards):
        sum = 0
        for card in cards:
            value = (card % 13) + 1
            if value > 10:
                value = 10
            sum += value
        return sum
    
    # initially deals player 2 cards and dealer 1 card and updates scores
    def deal_cards(self):
        random.shuffle(self.deck)
        self.state["myCards"].append(self.deck[0])
        self.deck.remove(self.deck[0])
        self.state["myCards"].append(self.deck[0])
        self.deck.remove(self.deck[0])
        self.state["dealerCards"].append(self.deck[0])
        self.deck.remove(self.deck[0])
        self.state['myScore'] = self.score(self.state['myCards'])
        self.state['dealerScore'] = self.score(self.state['dealerCards'])
    
    # converts card indices to playing cards with names
    def print_cards(self, cards):
        cardNames = []
        for card in cards:
            suit = self.cardSuits[math.floor(card / 13)]
            value = (card % 13) + 1
            if value in self.cardValues.keys():
                value = self.cardValues[value]
            card = str(value) + " of " + suit
            cardNames.append(card)
        return cardNames
    
    # prints out current state of the game
    def print_state(self):
        print("\n My Hand: ", end =" ")
        print(self.print_cards(self.state['myCards']), end =" ")
        print("My score: " + str(self.state['myScore']))
        print("Dealer's Hand: ", end =" ")
        print(self.print_cards(self.state['dealerCards']), end =" ")
        print("Dealer's Score: " + str(self.state['dealerScore']))
        if self.check_win() == 1:
            print("\n You win!")
        if self.check_win() == 2:
            print("\n You lose!")
        if self.check_win() == -1:
            print("\n You tie!")
            
    def generate_possible_hits(self):
        possible_hit_states = []
        current_state = copy.deepcopy(self.state)
        for i in range(0, len(self.deck)):
            current_state["myCards"].append(self.deck[i])
            current_state['myScore'] = self.score(current_state['myCards'])
            possible_hit_states.append(copy.deepcopy(current_state))
            current_state = copy.deepcopy(self.state)
        return possible_hit_states
        
            
    # hit (add cards to your hand) or stand (finish your turn and add cards to dealer's hand)
    def make_move(self, action):
        if self.winner != 0:
            return -1
        if action == "hit":
            self.state["myCards"].append(self.deck[0])
            self.deck.remove(self.deck[0])
            self.state['myScore'] = self.score(self.state['myCards'])
        elif action == "stand":
            while self.state['dealerScore'] < 17:
                self.state["dealerCards"].append(self.deck[0])
                self.deck.remove(self.deck[0])
                self.state['dealerScore'] = self.score(self.state['dealerCards'])
        return self.check_win()
    
    # check if the game has been won and update the winner accordingly
    def check_win(self):
        if self.state['myScore'] > 21:
            self.winner = 2
            return 2
        if len(self.state['dealerCards']) == 1:
            return 0
        elif self.state['dealerScore'] > 21:
            self.winner = 1
            return 1
        elif self.state['myScore'] > self.state['dealerScore']:
            self.winner = 1
            return 1
        elif self.state['dealerScore'] > self.state['myScore']:
            self.winner = 2
            return 2
        elif self.state['myScore'] == self.state['dealerScore']:
            self.winner = -1
            return -1
        return 0
    
    # a static method to check the status of a game
    @staticmethod
    def game_status(state):
        if state['myScore'] > 21:
            return 2
        if len(state['dealerCards']) == 1:
            return 0
        elif state['dealerScore'] > 21:
            return 1
        elif state['myScore'] > self.state['dealerScore']:
            return 1
        elif state['dealerScore'] > self.state['myScore']:
            return 2
        elif state['myScore'] == self.state['dealerScore']:
            return -1
        return 0


In [ ]:
# Play Example Game

print('\n ------------------------------------------------------------------------------------------ \n')
print('Turn 0: ')
cards = list(range(52))
game = BlackJack(cards)
game.deal_cards()
game.print_state()
print('\n ------------------------------------------------------------------------------------------ \n')

# Player hits once
game.make_move("hit")
print('Turn 1: Hit:')
game.print_state()
print('\n ------------------------------------------------------------------------------------------ \n')

# Player then stands
game.make_move("stand")
print('Turn 2: Stand:')
game.print_state()
print('\n ------------------------------------------------------------------------------------------ \n')

In [379]:
# the ML agent that will learn how to play BlackJack
class Agent:
    def __init__(self, learning_rate):
        # game states it has seen before and their corresponding values
        self.values = {}
        self.prev_state = None
        # how much the Agent should adjust values when finding good / bad states
        self.learning_rate = learning_rate
    
    # converts a state in dictionary form to tuple form
    def state_tuple(self, state):
        stateTuple = []
        stateTuple.append('myCards')
        for myCard in state['myCards']:
            stateTuple.append(myCard)
        stateTuple.append('dealerCards')
        for dealerCard in state['dealerCards']:
            stateTuple.append(dealerCard)
        stateTuple.extend(['myScore', state['myScore'], 'dealerScore', state['dealerScore']])
        return tuple(stateTuple)
        
    # converts a state in tuple form t dictionary form
    def state_dict(self, state):
        stateDict = {'myCards': [], 'dealerCards': [], 'myScore': 0, 'dealerScore': 0}
        stateTuple = list(state)
        stateTuple.pop(0)
        for i in range(0, stateTuple.index('dealerCards')):
            stateDict['myCards'].append(stateTuple[0])
            stateTuple.pop(0)
        stateTuple.pop(0)
        for i in range(0, stateTuple.index('myScore')):
            stateDict['dealerCards'].append(stateTuple[0])
            stateTuple.pop(0)
        stateTuple.pop(0)
        stateDict['myScore'] = stateTuple[0]
        stateTuple.pop(0)
        stateTuple.pop(0)
        stateDict['dealerScore'] = stateTuple[0]
        return stateDict
        
    def get_value(self, state):
        useState = state
        if(type(state) is dict):
            useState = self.state_tuple(state)
        if useState not in self.values.keys():
            self.values[useState] = 0.5
        return self.values[useState]
    
    def make_move(self, game, explore_prob=0.1):
        
        # check game state, and then update as needed
        if game.winner == 2:
            # Lost the game
            self.values[self.state_tuple(self.prev_state)] += self.learning_rate * (0 - self.get_value(self.prev_state))
            return -1
        elif game.winner == -1:
            # Tied the game
            self.values[self.state_tuple(self.prev_state)] += self.learning_rate * (0.5 - self.get_value(self.prev_state))
            return -1
        elif game.winner == 1:
            # Won the game
            self.values[self.state_tuple(self.prev_state)] += self.learning_rate * (1 - self.get_value(self.prev_state))
            return -1
        
        # Looking at possible moves
        possible_moves = ["hit", "stand"]
        possible_hit_states = game.generate_possible_hits()
        hit_vals = [self.get_value(state) for state in possible_hit_states]
        
        # Add some randomness to balance explore / exploit
        if np.random.random() < explore_prob:
            # Make exploratory move
            random_move = possible_moves[np.random.randint(len(possible_moves))]
            win = game.make_move(random_move)
            if win == 1:
                self.get_value(game.state)
                self.values[self.state_tuple(game.state)] += self.learning_rate * (1 - self.get_value(self.prev_state))
            elif win == 2:
                self.get_value(game.state)
                self.values[self.state_tuple(game.state)] += self.learning_rate * (0 - self.get_value(self.prev_state))
            # Don't update value of previous state since this is random
            self.prev_state = game.state
            return random_move
        else:
            # Make exploitative move;
            if sum(hit_vals) / len(hit_vals) >= self.get_value(game.state):
                win = game.make_move("hit")
            else:
                win = game.make_move("stand")
            if win == 1:
                self.get_value(game.state)
                self.values[self.state_tuple(game.state)] += self.learning_rate * (1 - self.get_value(self.prev_state))
            elif win == 2:
                self.get_value(game.state)
                self.values[self.state_tuple(game.state)] += self.learning_rate * (0 - self.get_value(self.prev_state))
            if self.prev_state is not None:
                # Update value of previous state
                self.get_value(self.prev_state)
                self.values[self.state_tuple(self.prev_state)] += self.learning_rate * (self.get_value(game.state) - self.get_value(self.prev_state))
            self.prev_state = game.state
            
            # return better_move
    
    def new_game(self):
        self.prev_state = None

In [381]:
# untrained tallies
wins = 0
ties = 0
losses = 0
total_games = 1000
cards = list(range(52))

# play total_games number of games, WITHOUT training / learning
for _ in range(total_games):
    game = BlackJack(cards)
    agent = Agent(0.05)
    agent.new_game()
    game.deal_cards()
    while True:
        agent.make_move(game, explore_prob=0.5)
        print(game.state)
        print("The winner is " + str(game.winner) + "\n")
        if game.winner != 0:
            break
    if game.winner == -1:
        ties += 1
    elif game.winner == 1:
        wins += 1
    elif game.winner == 2:
        losses += 1

# output the benchmark results
print(f"Record: {wins}-{losses}-{ties}")
print(f"Win Percentage: {100 * wins / total_games}")

{'myCards': [9, 41], 'dealerCards': [0, 35, 2, 42], 'myScore': 13, 'dealerScore': 18}
The winner is 2

{'myCards': [21, 42, 51], 'dealerCards': [26], 'myScore': 23, 'dealerScore': 1}
The winner is 2

{'myCards': [33, 0, 50], 'dealerCards': [8], 'myScore': 19, 'dealerScore': 9}
The winner is 0

{'myCards': [33, 0, 50, 21], 'dealerCards': [8], 'myScore': 28, 'dealerScore': 9}
The winner is 2

{'myCards': [41, 1, 48], 'dealerCards': [33], 'myScore': 15, 'dealerScore': 8}
The winner is 0

{'myCards': [41, 1, 48, 47], 'dealerCards': [33], 'myScore': 24, 'dealerScore': 8}
The winner is 2

{'myCards': [22, 51], 'dealerCards': [48, 5, 17], 'myScore': 20, 'dealerScore': 21}
The winner is 2

{'myCards': [18, 3, 46], 'dealerCards': [23], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [18, 3, 46, 4], 'dealerCards': [23], 'myScore': 23, 'dealerScore': 10}
The winner is 2

{'myCards': [6, 50], 'dealerCards': [45, 42, 24], 'myScore': 17, 'dealerScore': 21}
The winner is 2

{'myCards':

{'myCards': [41, 49, 38], 'dealerCards': [19], 'myScore': 23, 'dealerScore': 7}
The winner is 2

{'myCards': [14, 27, 21], 'dealerCards': [23], 'myScore': 13, 'dealerScore': 10}
The winner is 0

{'myCards': [14, 27, 21, 17], 'dealerCards': [23], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [14, 27, 21, 17], 'dealerCards': [23, 33], 'myScore': 18, 'dealerScore': 18}
The winner is -1

{'myCards': [25, 23, 6], 'dealerCards': [34], 'myScore': 27, 'dealerScore': 9}
The winner is 2

{'myCards': [36, 45], 'dealerCards': [4, 44, 19], 'myScore': 17, 'dealerScore': 18}
The winner is 2

{'myCards': [31, 39], 'dealerCards': [42, 21, 6], 'myScore': 7, 'dealerScore': 20}
The winner is 2

{'myCards': [27, 36, 16], 'dealerCards': [23], 'myScore': 16, 'dealerScore': 10}
The winner is 0

{'myCards': [27, 36, 16], 'dealerCards': [23, 22], 'myScore': 16, 'dealerScore': 20}
The winner is 2

{'myCards': [30, 15, 36], 'dealerCards': [33], 'myScore': 18, 'dealerScore': 8}
The winner is 0

{'

{'myCards': [5, 29, 13], 'dealerCards': [31], 'myScore': 11, 'dealerScore': 6}
The winner is 0

{'myCards': [5, 29, 13, 21], 'dealerCards': [31], 'myScore': 20, 'dealerScore': 6}
The winner is 0

{'myCards': [5, 29, 13, 21, 24], 'dealerCards': [31], 'myScore': 30, 'dealerScore': 6}
The winner is 2

{'myCards': [42, 39, 18], 'dealerCards': [21], 'myScore': 11, 'dealerScore': 9}
The winner is 0

{'myCards': [42, 39, 18], 'dealerCards': [21, 23], 'myScore': 11, 'dealerScore': 19}
The winner is 2

{'myCards': [2, 8, 49], 'dealerCards': [24], 'myScore': 22, 'dealerScore': 10}
The winner is 2

{'myCards': [40, 48, 43], 'dealerCards': [26], 'myScore': 17, 'dealerScore': 1}
The winner is 0

{'myCards': [40, 48, 43, 3], 'dealerCards': [26], 'myScore': 21, 'dealerScore': 1}
The winner is 0

{'myCards': [40, 48, 43, 3, 15], 'dealerCards': [26], 'myScore': 24, 'dealerScore': 1}
The winner is 2

{'myCards': [49, 37, 47], 'dealerCards': [9], 'myScore': 29, 'dealerScore': 10}
The winner is 2

{'myCar

{'myCards': [30, 37, 17], 'dealerCards': [51, 39, 20], 'myScore': 20, 'dealerScore': 19}
The winner is 1

{'myCards': [12, 49, 21], 'dealerCards': [43], 'myScore': 29, 'dealerScore': 5}
The winner is 2

{'myCards': [10, 2, 9], 'dealerCards': [19], 'myScore': 23, 'dealerScore': 7}
The winner is 2

{'myCards': [43, 22, 9], 'dealerCards': [15], 'myScore': 25, 'dealerScore': 3}
The winner is 2

{'myCards': [38, 7, 4], 'dealerCards': [35], 'myScore': 23, 'dealerScore': 10}
The winner is 2

{'myCards': [17, 5, 13], 'dealerCards': [43], 'myScore': 12, 'dealerScore': 5}
The winner is 0

{'myCards': [17, 5, 13, 39], 'dealerCards': [43], 'myScore': 13, 'dealerScore': 5}
The winner is 0

{'myCards': [17, 5, 13, 39], 'dealerCards': [43, 25, 44], 'myScore': 13, 'dealerScore': 21}
The winner is 2

{'myCards': [43, 18], 'dealerCards': [28, 0, 32, 21], 'myScore': 11, 'dealerScore': 20}
The winner is 2

{'myCards': [7, 18], 'dealerCards': [4, 19, 21], 'myScore': 14, 'dealerScore': 21}
The winner is 2



{'myCards': [13, 3, 2, 24], 'dealerCards': [5], 'myScore': 18, 'dealerScore': 6}
The winner is 0

{'myCards': [13, 3, 2, 24, 19], 'dealerCards': [5], 'myScore': 25, 'dealerScore': 6}
The winner is 2

{'myCards': [39, 8, 38], 'dealerCards': [23], 'myScore': 20, 'dealerScore': 10}
The winner is 0

{'myCards': [39, 8, 38, 28], 'dealerCards': [23], 'myScore': 23, 'dealerScore': 10}
The winner is 2

{'myCards': [37, 39], 'dealerCards': [1, 7, 29, 42], 'myScore': 11, 'dealerScore': 18}
The winner is 2

{'myCards': [40, 37, 43], 'dealerCards': [3], 'myScore': 17, 'dealerScore': 4}
The winner is 0

{'myCards': [40, 37, 43, 35], 'dealerCards': [3], 'myScore': 27, 'dealerScore': 4}
The winner is 2

{'myCards': [26, 48], 'dealerCards': [33, 37], 'myScore': 11, 'dealerScore': 18}
The winner is 2

{'myCards': [35, 21, 8], 'dealerCards': [39], 'myScore': 28, 'dealerScore': 1}
The winner is 2

{'myCards': [9, 37, 34], 'dealerCards': [22], 'myScore': 29, 'dealerScore': 10}
The winner is 2

{'myCards':

{'myCards': [3, 23], 'dealerCards': [37, 31, 0], 'myScore': 14, 'dealerScore': 17}
The winner is 2

{'myCards': [1, 17, 0], 'dealerCards': [37], 'myScore': 8, 'dealerScore': 10}
The winner is 0

{'myCards': [1, 17, 0, 5], 'dealerCards': [37], 'myScore': 14, 'dealerScore': 10}
The winner is 0

{'myCards': [1, 17, 0, 5, 13], 'dealerCards': [37], 'myScore': 15, 'dealerScore': 10}
The winner is 0

{'myCards': [1, 17, 0, 5, 13, 28], 'dealerCards': [37], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [1, 17, 0, 5, 13, 28], 'dealerCards': [37, 19], 'myScore': 18, 'dealerScore': 17}
The winner is 1

{'myCards': [1, 33, 2], 'dealerCards': [44], 'myScore': 13, 'dealerScore': 6}
The winner is 0

{'myCards': [1, 33, 2], 'dealerCards': [44, 21, 10], 'myScore': 13, 'dealerScore': 25}
The winner is 1

{'myCards': [31, 50, 45], 'dealerCards': [26], 'myScore': 23, 'dealerScore': 1}
The winner is 2

{'myCards': [51, 4, 33], 'dealerCards': [17], 'myScore': 23, 'dealerScore': 5}
The winner

{'myCards': [9, 24, 8], 'dealerCards': [2], 'myScore': 29, 'dealerScore': 3}
The winner is 2

{'myCards': [14, 46, 36], 'dealerCards': [4], 'myScore': 20, 'dealerScore': 5}
The winner is 0

{'myCards': [14, 46, 36, 43], 'dealerCards': [4], 'myScore': 25, 'dealerScore': 5}
The winner is 2

{'myCards': [20, 33], 'dealerCards': [22, 32], 'myScore': 16, 'dealerScore': 17}
The winner is 2

{'myCards': [49, 34, 17], 'dealerCards': [18], 'myScore': 24, 'dealerScore': 6}
The winner is 2

{'myCards': [5, 10], 'dealerCards': [35, 24], 'myScore': 16, 'dealerScore': 20}
The winner is 2

{'myCards': [10, 8, 46], 'dealerCards': [43], 'myScore': 27, 'dealerScore': 5}
The winner is 2

{'myCards': [37, 26], 'dealerCards': [5, 46, 14, 50], 'myScore': 11, 'dealerScore': 26}
The winner is 1

{'myCards': [51, 38, 28], 'dealerCards': [12], 'myScore': 23, 'dealerScore': 10}
The winner is 2

{'myCards': [4, 49, 50], 'dealerCards': [19], 'myScore': 25, 'dealerScore': 7}
The winner is 2

{'myCards': [1, 32, 13]

{'myCards': [32, 13, 12, 40], 'dealerCards': [47], 'myScore': 20, 'dealerScore': 9}
The winner is 0

{'myCards': [32, 13, 12, 40, 46], 'dealerCards': [47], 'myScore': 28, 'dealerScore': 9}
The winner is 2

{'myCards': [41, 25], 'dealerCards': [18, 33, 28], 'myScore': 13, 'dealerScore': 17}
The winner is 2

{'myCards': [33, 11, 49], 'dealerCards': [45], 'myScore': 28, 'dealerScore': 7}
The winner is 2

{'myCards': [13, 23, 7], 'dealerCards': [28], 'myScore': 19, 'dealerScore': 3}
The winner is 0

{'myCards': [13, 23, 7, 20], 'dealerCards': [28], 'myScore': 27, 'dealerScore': 3}
The winner is 2

{'myCards': [42, 37], 'dealerCards': [44, 1, 47], 'myScore': 14, 'dealerScore': 17}
The winner is 2

{'myCards': [26, 22, 24], 'dealerCards': [42], 'myScore': 21, 'dealerScore': 4}
The winner is 0

{'myCards': [26, 22, 24, 41], 'dealerCards': [42], 'myScore': 24, 'dealerScore': 4}
The winner is 2

{'myCards': [35, 46, 50], 'dealerCards': [8], 'myScore': 28, 'dealerScore': 9}
The winner is 2

{'my

{'myCards': [32, 17, 43, 41], 'dealerCards': [45], 'myScore': 20, 'dealerScore': 7}
The winner is 0

{'myCards': [32, 17, 43, 41, 8], 'dealerCards': [45], 'myScore': 29, 'dealerScore': 7}
The winner is 2

{'myCards': [9, 35], 'dealerCards': [37, 14, 13, 0, 3], 'myScore': 20, 'dealerScore': 18}
The winner is 1

{'myCards': [49, 31, 50], 'dealerCards': [9], 'myScore': 26, 'dealerScore': 10}
The winner is 2

{'myCards': [17, 29, 38], 'dealerCards': [50], 'myScore': 19, 'dealerScore': 10}
The winner is 0

{'myCards': [17, 29, 38], 'dealerCards': [50, 24], 'myScore': 19, 'dealerScore': 20}
The winner is 2

{'myCards': [30, 11], 'dealerCards': [8, 39, 16, 47], 'myScore': 15, 'dealerScore': 23}
The winner is 1

{'myCards': [3, 13, 46], 'dealerCards': [24], 'myScore': 13, 'dealerScore': 10}
The winner is 0

{'myCards': [3, 13, 46], 'dealerCards': [24, 26, 51], 'myScore': 13, 'dealerScore': 21}
The winner is 2

{'myCards': [24, 44, 32], 'dealerCards': [28], 'myScore': 23, 'dealerScore': 3}
The 

{'myCards': [26, 23, 27, 33], 'dealerCards': [50], 'myScore': 21, 'dealerScore': 10}
The winner is 0

{'myCards': [26, 23, 27, 33, 0], 'dealerCards': [50], 'myScore': 22, 'dealerScore': 10}
The winner is 2

Record: 176-803-21
Win Percentage: 17.6


In [382]:
# tallies with training
training_games = 10000
trained_agent = Agent(0.05)

# play training_games number of games, training the same Agent
for _ in range(total_games):
    game = BlackJack(cards)
    trained_agent.new_game()
    game.deal_cards()
    while True:
        print("My move is " + agent.make_move(game, explore_prob=1.0))
        print(game.state)
        print("The winner is " + str(game.winner) + "\n")
        if game.winner != 0:
            break
    if game.winner == -1:
        ties += 1
    elif game.winner == 1:
        wins += 1
    elif game.winner == 2:
        losses += 1

My move is stand
{'myCards': [25, 9], 'dealerCards': [49, 35], 'myScore': 20, 'dealerScore': 20}
The winner is -1

My move is hit
{'myCards': [45, 43, 33], 'dealerCards': [46], 'myScore': 20, 'dealerScore': 8}
The winner is 0

My move is stand
{'myCards': [45, 43, 33], 'dealerCards': [46, 26, 6, 40], 'myScore': 20, 'dealerScore': 18}
The winner is 1

My move is hit
{'myCards': [22, 40, 28], 'dealerCards': [10], 'myScore': 15, 'dealerScore': 10}
The winner is 0

My move is hit
{'myCards': [22, 40, 28, 47], 'dealerCards': [10], 'myScore': 24, 'dealerScore': 10}
The winner is 2

My move is stand
{'myCards': [18, 48], 'dealerCards': [23, 28, 38], 'myScore': 16, 'dealerScore': 23}
The winner is 1

My move is stand
{'myCards': [49, 27], 'dealerCards': [41, 20, 24], 'myScore': 12, 'dealerScore': 21}
The winner is 2

My move is stand
{'myCards': [11, 32], 'dealerCards': [40, 37, 9], 'myScore': 17, 'dealerScore': 22}
The winner is 1

My move is stand
{'myCards': [22, 3], 'dealerCards': [49, 40,

My move is hit
{'myCards': [47, 0, 20], 'dealerCards': [13], 'myScore': 18, 'dealerScore': 1}
The winner is 0

My move is stand
{'myCards': [47, 0, 20], 'dealerCards': [13, 9, 25], 'myScore': 18, 'dealerScore': 21}
The winner is 2

My move is hit
{'myCards': [0, 48, 8], 'dealerCards': [4], 'myScore': 20, 'dealerScore': 5}
The winner is 0

My move is stand
{'myCards': [0, 48, 8], 'dealerCards': [4, 27, 51], 'myScore': 20, 'dealerScore': 17}
The winner is 1

My move is stand
{'myCards': [5, 19], 'dealerCards': [18, 35, 3], 'myScore': 13, 'dealerScore': 20}
The winner is 2

My move is hit
{'myCards': [26, 25, 48], 'dealerCards': [12], 'myScore': 21, 'dealerScore': 10}
The winner is 0

My move is hit
{'myCards': [26, 25, 48, 21], 'dealerCards': [12], 'myScore': 30, 'dealerScore': 10}
The winner is 2

My move is hit
{'myCards': [21, 32, 51], 'dealerCards': [29], 'myScore': 26, 'dealerScore': 4}
The winner is 2

My move is hit
{'myCards': [29, 39, 12], 'dealerCards': [32], 'myScore': 15, 'de

My move is stand
{'myCards': [14, 47, 13, 32], 'dealerCards': [20, 28, 46], 'myScore': 19, 'dealerScore': 19}
The winner is -1

My move is stand
{'myCards': [49, 41], 'dealerCards': [6, 13, 3, 32], 'myScore': 13, 'dealerScore': 19}
The winner is 2

My move is hit
{'myCards': [12, 43, 40], 'dealerCards': [30], 'myScore': 17, 'dealerScore': 5}
The winner is 0

My move is hit
{'myCards': [12, 43, 40, 7], 'dealerCards': [30], 'myScore': 25, 'dealerScore': 5}
The winner is 2

My move is hit
{'myCards': [12, 3, 29], 'dealerCards': [45], 'myScore': 18, 'dealerScore': 7}
The winner is 0

My move is stand
{'myCards': [12, 3, 29], 'dealerCards': [45, 40, 26, 17, 44], 'myScore': 18, 'dealerScore': 21}
The winner is 2

My move is stand
{'myCards': [26, 1], 'dealerCards': [30, 37, 7], 'myScore': 3, 'dealerScore': 23}
The winner is 1

My move is hit
{'myCards': [46, 47, 12], 'dealerCards': [5], 'myScore': 27, 'dealerScore': 6}
The winner is 2

My move is hit
{'myCards': [29, 35, 27], 'dealerCards': 

The winner is 0

My move is stand
{'myCards': [18, 48, 40], 'dealerCards': [33, 35], 'myScore': 18, 'dealerScore': 18}
The winner is -1

My move is hit
{'myCards': [29, 28, 18], 'dealerCards': [11], 'myScore': 13, 'dealerScore': 10}
The winner is 0

My move is stand
{'myCards': [29, 28, 18], 'dealerCards': [11, 4, 35], 'myScore': 13, 'dealerScore': 25}
The winner is 1

My move is stand
{'myCards': [48, 28], 'dealerCards': [16, 11, 10], 'myScore': 13, 'dealerScore': 24}
The winner is 1

My move is stand
{'myCards': [4, 2], 'dealerCards': [6, 28, 33], 'myScore': 8, 'dealerScore': 18}
The winner is 2

My move is hit
{'myCards': [28, 43, 21], 'dealerCards': [25], 'myScore': 17, 'dealerScore': 10}
The winner is 0

My move is stand
{'myCards': [28, 43, 21], 'dealerCards': [25, 20], 'myScore': 17, 'dealerScore': 18}
The winner is 2

My move is stand
{'myCards': [14, 9], 'dealerCards': [27, 37, 3, 42], 'myScore': 12, 'dealerScore': 20}
The winner is 2

My move is stand
{'myCards': [41, 4], 'de

My move is hit
{'myCards': [9, 4, 5, 15], 'dealerCards': [23], 'myScore': 24, 'dealerScore': 10}
The winner is 2

My move is stand
{'myCards': [14, 8], 'dealerCards': [17, 50, 18], 'myScore': 11, 'dealerScore': 21}
The winner is 2

My move is hit
{'myCards': [8, 23, 32], 'dealerCards': [42], 'myScore': 26, 'dealerScore': 4}
The winner is 2

My move is stand
{'myCards': [18, 5], 'dealerCards': [39, 51, 12], 'myScore': 12, 'dealerScore': 21}
The winner is 2

My move is hit
{'myCards': [22, 26, 5], 'dealerCards': [45], 'myScore': 17, 'dealerScore': 7}
The winner is 0

My move is hit
{'myCards': [22, 26, 5, 1], 'dealerCards': [45], 'myScore': 19, 'dealerScore': 7}
The winner is 0

My move is hit
{'myCards': [22, 26, 5, 1, 38], 'dealerCards': [45], 'myScore': 29, 'dealerScore': 7}
The winner is 2

My move is hit
{'myCards': [8, 33, 14], 'dealerCards': [1], 'myScore': 19, 'dealerScore': 2}
The winner is 0

My move is stand
{'myCards': [8, 33, 14], 'dealerCards': [1, 47, 37], 'myScore': 19, '

My move is stand
{'myCards': [28, 3], 'dealerCards': [32, 17, 47], 'myScore': 7, 'dealerScore': 21}
The winner is 2

My move is stand
{'myCards': [18, 37], 'dealerCards': [49, 4, 21], 'myScore': 16, 'dealerScore': 24}
The winner is 1

My move is hit
{'myCards': [49, 40, 20], 'dealerCards': [29], 'myScore': 20, 'dealerScore': 4}
The winner is 0

My move is stand
{'myCards': [49, 40, 20], 'dealerCards': [29, 38, 8], 'myScore': 20, 'dealerScore': 23}
The winner is 1

My move is stand
{'myCards': [21, 40], 'dealerCards': [17, 10, 18], 'myScore': 11, 'dealerScore': 21}
The winner is 2

My move is stand
{'myCards': [10, 22], 'dealerCards': [0, 48, 8], 'myScore': 20, 'dealerScore': 20}
The winner is -1

My move is stand
{'myCards': [49, 25], 'dealerCards': [50, 23], 'myScore': 20, 'dealerScore': 20}
The winner is -1

My move is hit
{'myCards': [8, 28, 50], 'dealerCards': [49], 'myScore': 22, 'dealerScore': 10}
The winner is 2

My move is hit
{'myCards': [39, 6, 11], 'dealerCards': [1], 'mySco

My move is hit
{'myCards': [11, 26, 45], 'dealerCards': [32], 'myScore': 18, 'dealerScore': 7}
The winner is 0

My move is stand
{'myCards': [11, 26, 45], 'dealerCards': [32, 42, 14, 22], 'myScore': 18, 'dealerScore': 23}
The winner is 1

My move is stand
{'myCards': [48, 18], 'dealerCards': [32, 21, 28], 'myScore': 16, 'dealerScore': 19}
The winner is 2

My move is hit
{'myCards': [16, 37, 5], 'dealerCards': [30], 'myScore': 20, 'dealerScore': 5}
The winner is 0

My move is hit
{'myCards': [16, 37, 5, 3], 'dealerCards': [30], 'myScore': 24, 'dealerScore': 5}
The winner is 2

My move is hit
{'myCards': [9, 22, 8], 'dealerCards': [34], 'myScore': 29, 'dealerScore': 9}
The winner is 2

My move is stand
{'myCards': [33, 14], 'dealerCards': [23, 17, 24], 'myScore': 10, 'dealerScore': 25}
The winner is 1

My move is stand
{'myCards': [6, 14], 'dealerCards': [18, 28, 4, 7], 'myScore': 9, 'dealerScore': 22}
The winner is 1

My move is hit
{'myCards': [30, 11, 16], 'dealerCards': [50], 'myScor

My move is stand
{'myCards': [22, 39], 'dealerCards': [0, 27, 38, 25], 'myScore': 11, 'dealerScore': 23}
The winner is 1

My move is hit
{'myCards': [6, 27, 48], 'dealerCards': [37], 'myScore': 19, 'dealerScore': 10}
The winner is 0

My move is stand
{'myCards': [6, 27, 48], 'dealerCards': [37, 41, 40, 15], 'myScore': 19, 'dealerScore': 18}
The winner is 1

My move is stand
{'myCards': [9, 11], 'dealerCards': [36, 40, 0, 33], 'myScore': 20, 'dealerScore': 21}
The winner is 2

My move is hit
{'myCards': [16, 50, 1], 'dealerCards': [46], 'myScore': 16, 'dealerScore': 8}
The winner is 0

My move is hit
{'myCards': [16, 50, 1, 37], 'dealerCards': [46], 'myScore': 26, 'dealerScore': 8}
The winner is 2

My move is stand
{'myCards': [36, 27], 'dealerCards': [14, 43, 18, 21], 'myScore': 12, 'dealerScore': 22}
The winner is 1

My move is hit
{'myCards': [37, 35, 51], 'dealerCards': [47], 'myScore': 30, 'dealerScore': 9}
The winner is 2

My move is hit
{'myCards': [32, 9, 23], 'dealerCards': [17

My move is hit
{'myCards': [19, 12, 40], 'dealerCards': [32], 'myScore': 19, 'dealerScore': 7}
The winner is 0

My move is hit
{'myCards': [19, 12, 40, 36], 'dealerCards': [32], 'myScore': 29, 'dealerScore': 7}
The winner is 2

My move is hit
{'myCards': [4, 26, 43], 'dealerCards': [44], 'myScore': 11, 'dealerScore': 6}
The winner is 0

My move is stand
{'myCards': [4, 26, 43], 'dealerCards': [44, 47, 5], 'myScore': 11, 'dealerScore': 21}
The winner is 2

My move is stand
{'myCards': [50, 11], 'dealerCards': [2, 5, 28, 27, 23], 'myScore': 20, 'dealerScore': 24}
The winner is 1

My move is hit
{'myCards': [35, 5, 43], 'dealerCards': [32], 'myScore': 21, 'dealerScore': 7}
The winner is 0

My move is hit
{'myCards': [35, 5, 43, 12], 'dealerCards': [32], 'myScore': 31, 'dealerScore': 7}
The winner is 2

My move is hit
{'myCards': [38, 24, 0], 'dealerCards': [48], 'myScore': 21, 'dealerScore': 10}
The winner is 0

My move is stand
{'myCards': [38, 24, 0], 'dealerCards': [48, 13, 8], 'myScor

In [384]:
# untrained tallies
wins = 0
ties = 0
losses = 0
total_games = 1000
cards = list(range(52))

# play total_games number of games, WITHOUT training / learning
for _ in range(total_games):
    game = BlackJack(cards)
    trained_agent.new_game()
    game.deal_cards()
    while True:
        agent.make_move(game, explore_prob=0.5)
        print(game.state)
        print("The winner is " + str(game.winner) + "\n")
        if game.winner != 0:
            break
    if game.winner == -1:
        ties += 1
    elif game.winner == 1:
        wins += 1
    elif game.winner == 2:
        losses += 1

# output the benchmark results
print(f"Record: {wins}-{losses}-{ties}")
print(f"Win Percentage: {100 * wins / total_games}")

{'myCards': [24, 34], 'dealerCards': [37, 42, 27, 26], 'myScore': 19, 'dealerScore': 17}
The winner is 1

{'myCards': [8, 7, 18], 'dealerCards': [2], 'myScore': 23, 'dealerScore': 3}
The winner is 2

{'myCards': [36, 31, 10], 'dealerCards': [30], 'myScore': 26, 'dealerScore': 5}
The winner is 2

{'myCards': [44, 27], 'dealerCards': [25, 35], 'myScore': 8, 'dealerScore': 20}
The winner is 2

{'myCards': [12, 22, 41], 'dealerCards': [0], 'myScore': 23, 'dealerScore': 1}
The winner is 2

{'myCards': [37, 14], 'dealerCards': [39, 31, 46, 22], 'myScore': 12, 'dealerScore': 25}
The winner is 1

{'myCards': [2, 29, 13], 'dealerCards': [22], 'myScore': 8, 'dealerScore': 10}
The winner is 0

{'myCards': [2, 29, 13], 'dealerCards': [22, 8], 'myScore': 8, 'dealerScore': 19}
The winner is 2

{'myCards': [30, 8], 'dealerCards': [38, 51], 'myScore': 14, 'dealerScore': 20}
The winner is 2

{'myCards': [48, 29, 3], 'dealerCards': [40], 'myScore': 18, 'dealerScore': 2}
The winner is 0

{'myCards': [48,

{'myCards': [27, 32, 29], 'dealerCards': [41], 'myScore': 13, 'dealerScore': 3}
The winner is 0

{'myCards': [27, 32, 29], 'dealerCards': [41, 8, 34], 'myScore': 13, 'dealerScore': 21}
The winner is 2

{'myCards': [49, 13, 32], 'dealerCards': [51], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [49, 13, 32, 11], 'dealerCards': [51], 'myScore': 28, 'dealerScore': 10}
The winner is 2

{'myCards': [24, 25], 'dealerCards': [4, 33, 50], 'myScore': 20, 'dealerScore': 23}
The winner is 1

{'myCards': [49, 41, 31], 'dealerCards': [44], 'myScore': 19, 'dealerScore': 6}
The winner is 0

{'myCards': [49, 41, 31, 20], 'dealerCards': [44], 'myScore': 27, 'dealerScore': 6}
The winner is 2

{'myCards': [21, 36, 33], 'dealerCards': [39], 'myScore': 27, 'dealerScore': 1}
The winner is 2

{'myCards': [51, 16, 8], 'dealerCards': [7], 'myScore': 23, 'dealerScore': 8}
The winner is 2

{'myCards': [24, 21, 20], 'dealerCards': [46], 'myScore': 27, 'dealerScore': 8}
The winner is 2

{'myCards'

{'myCards': [19, 0, 36], 'dealerCards': [23], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [19, 0, 36], 'dealerCards': [23, 11], 'myScore': 18, 'dealerScore': 20}
The winner is 2

{'myCards': [35, 50, 30], 'dealerCards': [26], 'myScore': 25, 'dealerScore': 1}
The winner is 2

{'myCards': [17, 1, 15], 'dealerCards': [49], 'myScore': 10, 'dealerScore': 10}
The winner is 0

{'myCards': [17, 1, 15, 7], 'dealerCards': [49], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [17, 1, 15, 7, 39], 'dealerCards': [49], 'myScore': 19, 'dealerScore': 10}
The winner is 0

{'myCards': [17, 1, 15, 7, 39], 'dealerCards': [49, 11], 'myScore': 19, 'dealerScore': 20}
The winner is 2

{'myCards': [16, 30, 39], 'dealerCards': [2], 'myScore': 10, 'dealerScore': 3}
The winner is 0

{'myCards': [16, 30, 39], 'dealerCards': [2, 29, 26, 49], 'myScore': 10, 'dealerScore': 18}
The winner is 2

{'myCards': [0, 25, 10], 'dealerCards': [46], 'myScore': 21, 'dealerScore': 8}
The winner i

{'myCards': [3, 32, 23], 'dealerCards': [4], 'myScore': 21, 'dealerScore': 5}
The winner is 0

{'myCards': [3, 32, 23, 19], 'dealerCards': [4], 'myScore': 28, 'dealerScore': 5}
The winner is 2

{'myCards': [15, 24, 31], 'dealerCards': [38], 'myScore': 19, 'dealerScore': 10}
The winner is 0

{'myCards': [15, 24, 31, 14], 'dealerCards': [38], 'myScore': 21, 'dealerScore': 10}
The winner is 0

{'myCards': [15, 24, 31, 14, 6], 'dealerCards': [38], 'myScore': 28, 'dealerScore': 10}
The winner is 2

{'myCards': [2, 21, 14], 'dealerCards': [37], 'myScore': 14, 'dealerScore': 10}
The winner is 0

{'myCards': [2, 21, 14, 10], 'dealerCards': [37], 'myScore': 24, 'dealerScore': 10}
The winner is 2

{'myCards': [28, 26, 16], 'dealerCards': [44], 'myScore': 8, 'dealerScore': 6}
The winner is 0

{'myCards': [28, 26, 16, 35], 'dealerCards': [44], 'myScore': 18, 'dealerScore': 6}
The winner is 0

{'myCards': [28, 26, 16, 35, 0], 'dealerCards': [44], 'myScore': 19, 'dealerScore': 6}
The winner is 0

{'

{'myCards': [4, 15], 'dealerCards': [36, 21], 'myScore': 8, 'dealerScore': 19}
The winner is 2

{'myCards': [0, 39, 51], 'dealerCards': [42], 'myScore': 12, 'dealerScore': 4}
The winner is 0

{'myCards': [0, 39, 51, 7], 'dealerCards': [42], 'myScore': 20, 'dealerScore': 4}
The winner is 0

{'myCards': [0, 39, 51, 7, 40], 'dealerCards': [42], 'myScore': 22, 'dealerScore': 4}
The winner is 2

{'myCards': [26, 30, 35], 'dealerCards': [15], 'myScore': 16, 'dealerScore': 3}
The winner is 0

{'myCards': [26, 30, 35, 5], 'dealerCards': [15], 'myScore': 22, 'dealerScore': 3}
The winner is 2

{'myCards': [33, 4], 'dealerCards': [7, 47], 'myScore': 13, 'dealerScore': 17}
The winner is 2

{'myCards': [19, 7, 45], 'dealerCards': [17], 'myScore': 22, 'dealerScore': 5}
The winner is 2

{'myCards': [13, 5, 22], 'dealerCards': [46], 'myScore': 17, 'dealerScore': 8}
The winner is 0

{'myCards': [13, 5, 22, 39], 'dealerCards': [46], 'myScore': 18, 'dealerScore': 8}
The winner is 0

{'myCards': [13, 5, 2

...

{'myCards': [20, 44], 'dealerCards': [16, 15, 43, 17], 'myScore': 14, 'dealerScore': 17}
The winner is 2

{'myCards': [41, 46, 0], 'dealerCards': [10], 'myScore': 12, 'dealerScore': 10}
The winner is 0

{'myCards': [41, 46, 0, 13], 'dealerCards': [10], 'myScore': 13, 'dealerScore': 10}
The winner is 0

{'myCards': [41, 46, 0, 13, 15], 'dealerCards': [10], 'myScore': 16, 'dealerScore': 10}
The winner is 0

{'myCards': [41, 46, 0, 13, 15, 8], 'dealerCards': [10], 'myScore': 25, 'dealerScore': 10}
The winner is 2

{'myCards': [0, 45, 40], 'dealerCards': [23], 'myScore': 10, 'dealerScore': 10}
The winner is 0

{'myCards': [0, 45, 40, 30], 'dealerCards': [23], 'myScore': 15, 'dealerScore': 10}
The winner is 0

{'myCards': [0, 45, 40, 30], 'dealerCards': [23, 27, 16, 4], 'myScore': 15, 'dealerScore': 21}
The winner is 2

{'myCards': [2, 5, 12], 'dealerCards': [39], 'myScore': 19, 'dealerScore': 1}
The winner is 0

{'myCards': [2, 5, 12, 22], 'dealerCards': [39], 'myScore': 29, 'dealerScore':

{'myCards': [45, 33, 26], 'dealerCards': [40], 'myScore': 16, 'dealerScore': 2}
The winner is 0

{'myCards': [45, 33, 26, 42], 'dealerCards': [40], 'myScore': 20, 'dealerScore': 2}
The winner is 0

{'myCards': [45, 33, 26, 42], 'dealerCards': [40, 39, 11, 34], 'myScore': 20, 'dealerScore': 22}
The winner is 1

{'myCards': [39, 33, 6], 'dealerCards': [23], 'myScore': 16, 'dealerScore': 10}
The winner is 0

{'myCards': [39, 33, 6, 12], 'dealerCards': [23], 'myScore': 26, 'dealerScore': 10}
The winner is 2

{'myCards': [39, 34, 22], 'dealerCards': [24], 'myScore': 20, 'dealerScore': 10}
The winner is 0

{'myCards': [39, 34, 22, 11], 'dealerCards': [24], 'myScore': 30, 'dealerScore': 10}
The winner is 2

{'myCards': [46, 51], 'dealerCards': [30, 47, 37], 'myScore': 18, 'dealerScore': 24}
The winner is 1

{'myCards': [39, 36, 26], 'dealerCards': [19], 'myScore': 12, 'dealerScore': 7}
The winner is 0

{'myCards': [39, 36, 26, 0], 'dealerCards': [19], 'myScore': 13, 'dealerScore': 7}
The winn

{'myCards': [8, 44], 'dealerCards': [30, 12, 9], 'myScore': 15, 'dealerScore': 25}
The winner is 1

{'myCards': [8, 48, 16], 'dealerCards': [24], 'myScore': 23, 'dealerScore': 10}
The winner is 2

{'myCards': [10, 22, 8], 'dealerCards': [3], 'myScore': 29, 'dealerScore': 4}
The winner is 2

{'myCards': [51, 18, 39], 'dealerCards': [49], 'myScore': 17, 'dealerScore': 10}
The winner is 0

{'myCards': [51, 18, 39], 'dealerCards': [49, 25], 'myScore': 17, 'dealerScore': 20}
The winner is 2

{'myCards': [35, 15, 14], 'dealerCards': [49], 'myScore': 15, 'dealerScore': 10}
The winner is 0

{'myCards': [35, 15, 14, 34], 'dealerCards': [49], 'myScore': 24, 'dealerScore': 10}
The winner is 2

{'myCards': [30, 7, 9], 'dealerCards': [21], 'myScore': 23, 'dealerScore': 9}
The winner is 2

{'myCards': [34, 50, 9], 'dealerCards': [30], 'myScore': 29, 'dealerScore': 5}
The winner is 2

{'myCards': [4, 45, 18], 'dealerCards': [10], 'myScore': 18, 'dealerScore': 10}
The winner is 0

{'myCards': [4, 45, 

The winner is 2

{'myCards': [3, 13, 40], 'dealerCards': [35], 'myScore': 7, 'dealerScore': 10}
The winner is 0

{'myCards': [3, 13, 40, 29], 'dealerCards': [35], 'myScore': 11, 'dealerScore': 10}
The winner is 0

{'myCards': [3, 13, 40, 29, 8], 'dealerCards': [35], 'myScore': 20, 'dealerScore': 10}
The winner is 0

{'myCards': [3, 13, 40, 29, 8, 0], 'dealerCards': [35], 'myScore': 21, 'dealerScore': 10}
The winner is 0

{'myCards': [3, 13, 40, 29, 8, 0, 42], 'dealerCards': [35], 'myScore': 25, 'dealerScore': 10}
The winner is 2

{'myCards': [38, 41, 22], 'dealerCards': [31], 'myScore': 23, 'dealerScore': 6}
The winner is 2

{'myCards': [3, 36, 11], 'dealerCards': [7], 'myScore': 24, 'dealerScore': 8}
The winner is 2

{'myCards': [5, 9, 45], 'dealerCards': [42], 'myScore': 23, 'dealerScore': 4}
The winner is 2

{'myCards': [16, 4], 'dealerCards': [37, 36], 'myScore': 9, 'dealerScore': 20}
The winner is 2

{'myCards': [17, 10, 18], 'dealerCards': [28], 'myScore': 21, 'dealerScore': 3}
T

{'myCards': [45, 43, 12], 'dealerCards': [7], 'myScore': 22, 'dealerScore': 8}
The winner is 2

{'myCards': [27, 16, 38], 'dealerCards': [45], 'myScore': 16, 'dealerScore': 7}
The winner is 0

{'myCards': [27, 16, 38, 2], 'dealerCards': [45], 'myScore': 19, 'dealerScore': 7}
The winner is 0

{'myCards': [27, 16, 38, 2, 37], 'dealerCards': [45], 'myScore': 29, 'dealerScore': 7}
The winner is 2

{'myCards': [15, 14, 25], 'dealerCards': [44], 'myScore': 15, 'dealerScore': 6}
The winner is 0

{'myCards': [15, 14, 25, 40], 'dealerCards': [44], 'myScore': 17, 'dealerScore': 6}
The winner is 0

{'myCards': [15, 14, 25, 40, 16], 'dealerCards': [44], 'myScore': 21, 'dealerScore': 6}
The winner is 0

{'myCards': [15, 14, 25, 40, 16, 6], 'dealerCards': [44], 'myScore': 28, 'dealerScore': 6}
The winner is 2

{'myCards': [27, 25, 13], 'dealerCards': [34], 'myScore': 13, 'dealerScore': 9}
The winner is 0

{'myCards': [27, 25, 13, 31], 'dealerCards': [34], 'myScore': 19, 'dealerScore': 9}
The winner 